Tracking Algorithms:
- ByteTrack
- Sort Tracker
- DeepSort

- FairMOT
- TrackFormer
- StrongSORT


In [ ]:
#imports and paths
import os
from inference import get_model
import supervision as sv
from ultralytics import YOLO
import numpy as np

VIDEO_PATH = "../../data/videos/old10s.mp4"
YOLO_PATH = "../../utils/models/yolov8n.pt"
PLAYER_DETECTION_MODEL_ID = 'football-players-detection-3zvbc/2'
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")
PLAYER_DETECTION_MODEL = get_model(PLAYER_DETECTION_MODEL_ID, ROBOFLOW_API_KEY)


ByteTracker

In [ ]:
#without team assignment & gk assignment
tracker = sv.ByteTrack() #supervision's implementation of ByteTrack

bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

def callback(frame: np.ndarray, index: int) -> np.ndarray:
    results = PLAYER_DETECTION_MODEL.infer(frame,confidence=0.3)[0]
    detections = sv.Detections.from_inference(results)
    
    detections = tracker.update_with_detections(detections)

    labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]

    annotated_frame = bounding_box_annotator.annotate(
        scene=frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels)
    return annotated_frame

sv.process_video(
    source_path= VIDEO_PATH,
    target_path= "../../data/tracker_outputs/bytetracker.mp4",
    callback=callback
)